In [ ]:
from PyQt6.QtWidgets import QApplication, QFileDialog
import sys

def select_folder_pyqt6():
    # Create application
    app = QApplication(sys.argv)

    # Open folder dialog
    folder_path = QFileDialog.getExistingDirectory(
        None,
        "Select a folder",
        ""
    )

    # Close QApplication
    app.exit()

    return folder_path

path = select_folder_pyqt6()
print(f"path: {path}")

### Select fitting strategy

In [ ]:
from shg_analysis import SHGDataAnalysis
from crystaldatabase import CRYSTALS
from crystaldatabase import *

analysis = SHGDataAnalysis(path)
meta = analysis.meta
data = analysis.data

crystal = CRYSTALS[meta["material"]]()

if crystal.axiality == "uniaxial":
    from fitting_strategies.jerphagnon1970 import Jerphagnon1970Strategy
    strategy = Jerphagnon1970Strategy(analysis)

elif crystal.axiality == "biaxial":
    from fitting_strategies.ishidate1974 import Ishidate1974Strategy
    strategy = Ishidate1974Strategy(analysis)

In [ ]:
fringe = strategy._maker_fringes(override={"theta_deg":data["position"]})
envelope = strategy._maker_fringes(override={"theta_deg":data["position"]}, envelope=True)

x = analysis.data["position"]
intensity_corrected = analysis.data["intensity_corrected"]

### theoretical fringe before fitting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
ax.plot(x, fringe, label='Maker fringe')
ax.plot(x, envelope, label='Maker fringe envelope')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
if np.max(envelope) > 10.0:
      ax.set_ylim(0, 10)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()

In [ ]:
plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(x, envelope, label='Maker fringe envelope')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)
# ax.set_yscale("log")

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()

In [ ]:
try:
    results = strategy.fit_all()
except Exception as e:
    print(f"Error:{e}")
# results = strategy.fit_all()

### Offset subtraction

In [ ]:
_, fit_offset = strategy._subtract_offset(analysis.data)
minima_x = x[fit_offset["minima_idx"]]
minima_y = intensity_corrected[fit_offset["minima_idx"]]
offset = fit_offset["offset"]
print("offset: ",offset)


plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
ax.plot(minima_x, minima_y, "*", ms=10, label='minima peaks')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
# ax.set_ylim(-0.1, 3.2)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()


In [ ]:
print(minima_x)

### Position centering fit

In [ ]:
_, fit_centering = strategy._position_centering(analysis.data)
center_x = fit_centering["c_candidates"]
center_cost = fit_centering["costs"]
center = fit_centering["c_best"]
print("center position: ",center)


plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
ax.plot(center_x, center_cost, label='centering cost')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
# ax.set_ylim(0, 3.2)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()


### sample thickness fitting

In [ ]:
L_fit = strategy._fit_L_small_angle(meta, analysis.data)
print(f"measured thickness:{analysis.meta["thickness_info"]["t_at_thin_end_mm"]}")
print(L_fit)

L_override = L_fit["L_mm"]

In [ ]:
plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

L_fitted_fringe = strategy._maker_fringes(override={"L":L_override, "theta_deg": x - center})
L_fitted_envelope = strategy._maker_fringes(envelope=True, override={"L":L_override, "theta_deg": x - center})

ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
ax.plot(x, L_fitted_fringe, label='t,center-adjusted maker fringe')
ax.plot(x, L_fitted_envelope, label='t,center-adjusted envelope')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
if np.max(L_fitted_envelope) > 15.0:
      ax.set_ylim(0, 12)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()

### Calculate Lc

In [ ]:
try:
    lc_results, fit_Lc = strategy._calc_Lc_large_angle(analysis.meta, analysis.data, [0,180], L_fit["L_mm"])
   
    minima_x = x[fit_Lc["minima_idx"]]
    minima_y = intensity_corrected[fit_Lc["minima_idx"]]

    theta_in_range = x[fit_Lc["x_in_range"]]
    y = [1 for i in theta_in_range]


    plt.rcParams['font.size'] = 10
    fig, ax = plt.subplots(figsize=(12, 4))

    ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
    ax.plot(minima_x, minima_y, "*", ms=10, label='minima peaks')
    ax.plot(theta_in_range, y, "*")

    ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
    ax.set_ylabel('Intensity (a.u.)', fontsize=12)

    # ax.set_xlim(-20, 20)
    # ax.set_ylim(-0.1, 3.2)

    fig.tight_layout()
    ax.legend(loc='upper left')
    plt.show()

    print(f"pos side:{fit_Lc["dL_pos"]}\n")
    print(f"neg side:{fit_Lc["dL_neg"]}\n")
    print(f"Lc: {lc_results["Lc_mean_mm"]*1000} ± {lc_results["Lc_std_mm"]*1000} um")

except Exception as e:
    print(f"Error during '_calc_Lc_large_angle':{e}")


### Calculate Lc manually

In [ ]:
wl1_nm = meta["wavelength_nm"]
wl1_mm = wl1_nm * 1e-6
pol_in = meta["input_polarization"] # 0-90 deg
pol_out = meta["detected_polarization"] # 0-90 deg

fitted_L_mm = L_fit["L_mm"]

def differential_L(th_list):
    if th_list.size < 2:
        return np.array([], dtype=float)
    
    th_rad = np.radians(th_list)
    lc_list = []
    for i in range(th_rad.size - 1):
        # Use midpoint refractive indices between adjacent angles
        n_w = strategy.n_eff(pol_in, wl1_nm, th_list[i])
        n_2w = strategy.n_eff(pol_out, wl1_nm / 2.0, th_list[i])

        lc = fitted_L_mm * (np.sin(th_rad[i + 1])**2 - np.sin(th_rad[i])**2) / (4.0 * n_2w * n_w)
        lc_list.append(abs(lc))

    return np.asarray(lc_list, dtype=float)

In [ ]:
theta_list_lc = np.array([7, 14.4])
lc = differential_L(theta_list_lc)

print(f"Lc: {lc}")

### Intensity fit

In [ ]:
fit_results, fit_Pm0 = strategy._fit_Pm0(analysis.data)
maxima_x = x[fit_Pm0["maxima_idx"]]
maxima_y = intensity_corrected[fit_Pm0["maxima_idx"]]


plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(x, intensity_corrected, label='intensity_corrected data', linestyle='--')
ax.plot(maxima_x, maxima_y, "*", ms=10, label='maxima peaks')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
# ax.set_ylim(0.0, 4)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()


In [ ]:
k = fit_results["Pm0"]
L = L_fit["L_mm"]
position_centered = strategy.analysis.data["position_centered"]

fitted_fringe = k * strategy._maker_fringes(override={"L":L, "theta_deg": position_centered})
fitted_envelope = k * strategy._maker_fringes(envelope=True, override={"L":L, "theta_deg": position_centered})

plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(position_centered, intensity_corrected - offset, label='Maker fringe', linestyle='--')
ax.plot(position_centered, fitted_fringe, label='Maker fringe')
ax.plot(position_centered, fitted_envelope, label='Envelope')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=18)
ax.set_ylabel('Intensity (a.u.)', fontsize=18)

ax.tick_params(axis="both", labelsize=18)

# ax.set_xlim(-20, 20)
if np.max(fitted_envelope) > 10.0:
      ax.set_ylim(0, np.max(intensity_corrected)*1.1)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()

### Change L manually for better fitting

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---- Plot style control (single variable) ----
ft = 16  # Change this to scale everything

plt.rcParams.update({
    "font.size": ft,
    "axes.titlesize": ft,
    "axes.labelsize": ft,
    "xtick.labelsize": ft,
    "ytick.labelsize": ft,
    "legend.fontsize": ft,
})

# ---- Angle axis ----
theta_deg = position_centered

# ---- L settings ----
L0_mm = analysis.meta["thickness_info"]["t_at_thin_end_mm"]  # <-- set your nominal L here [mm]
dL_um = 50.0   # +/- range [um]
dL_mm = dL_um / 1000.0

# ---- Vertical scale settings ----
A0 = fit_results["Pm0"]          # Initial vertical scale
A_range = np.max(intensity_corrected) * 1.5


slider_dL = widgets.FloatSlider(
    value=0.0,
    min=-dL_um,
    max=+dL_um,
    step=dL_um / 300.0,
    description="ΔL (µm)",
    continuous_update=True,
    readout=True,
    readout_format=".1f",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="520px"),
)

slider_A = widgets.FloatSlider(
    value=A0,
    min=0,
    max=A_range,
    step=A_range / 300.0,
    description="Scale A",
    continuous_update=True,
    readout=True,
    readout_format=".2f",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="520px"),
)

btn_reset = widgets.Button(description="Reset", layout=widgets.Layout(width="120px"))

out = widgets.Output()

def redraw(dL_current_um: float, A: float):
    L_mm = L0_mm + dL_current_um / 1000.0
    fringe_manual = A * strategy._maker_fringes(override={"L":L_mm, "theta_deg": position_centered}) \
      + offset
    envelope_manual = A * strategy._maker_fringes(override={"L":L_mm, "theta_deg": position_centered}, envelope=True) \
      + offset

    with out:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(theta_deg, fringe_manual, lw=2, label="Maker fringe (model)")
        ax.plot(theta_deg, envelope_manual, lw=2, label="Maker fringe (model)")
        ax.plot(theta_deg, intensity_corrected, label='intensity_corrected data', linestyle='--')
        ax.set_title("Maker fringe vs incidence angle")
        ax.set_xlabel("Incidence angle (deg)")
        ax.set_ylabel("Normalized intensity (a.u.)")
        # ax.set_xlim(theta_min, theta_max)
        # ax.set_ylim(-0.05, 1.05)
        if np.max(fitted_envelope) > 10.0:
            ax.set_ylim(0, np.max(intensity_corrected) * 1.1)
        ax.grid(True)
        ax.text(
            0.02, 0.95,
            f"L = {L_mm:.6f} mm  (ΔL = {dL_current_um:+.1f} µm)",
            transform=ax.transAxes,
            va="top",
            ha="left",
        )
        ax.legend(loc="upper right")
        plt.show()

def _on_slider_change(_=None):
    redraw(slider_dL.value, slider_A.value)


def _on_reset(_):
    slider_dL.value = 0.0
    slider_A.value = 1.0

slider_dL.observe(lambda ch: _on_slider_change(), names="value")
slider_A.observe(lambda ch: _on_slider_change(), names="value")
btn_reset.on_click(_on_reset)

display(widgets.VBox([
    widgets.HBox([slider_dL, btn_reset]),
    slider_A,
    out
]))

# Initial draw
redraw(slider_dL.value, slider_A.value)

#### Note
20251227 quartz-1 d11
L =0.5033 (-7.7 um)
A = 2.60

20251227 KDP-1 d36
L=0.2438
A = 1.82

### Comparison with reference

In [ ]:
# KDP vs quartz

# fit, aux = strategy._maker_fringes(override={"L":L, "theta_deg": position_centered}, return_aux=True)
# quartz d11 sensitivity=1/0.1V  OD=1+2
# OD1.0 (#1) transmission was 11.757399%
d_factor_quartzd11 = 366.5492256764717
peak_quartz = 2.60
d_propto_quartz = np.sqrt(0.1 * (1/0.1176) *peak_quartz / d_factor_quartzd11)

# quartz d11 sensitivity=1/0.2V  OD=1+2
peak_quartz_wedge = 1.29
d_propto_quartz_wedge = np.sqrt(0.2 * (1/0.1176) *peak_quartz_wedge / d_factor_quartzd11)

# KDP d36
peak_KDP = 1.82
d_factor_KDP = 118.48921149685113
d_propto_KDP = np.sqrt(0.1 * (1/0.1176) * peak_KDP / d_factor_KDP)

quartz_KDP = d_propto_quartz / d_propto_KDP 

d_abs_KDP = 0.43
d_abs_quartz = 0.30 # d11 from shoji, et.al.
d_rel_quartz = d_abs_KDP * quartz_KDP

print(f"KDP_vs_quartz: 1:{quartz_KDP}")
print(f"d_11_quartz{d_rel_quartz}")

In [ ]:
# path = select_folder_pyqt6()
# print(f"path: {path}")


# analysis_ref = SHGDataAnalysis(path)
# meta_ref = analysis_ref.meta
# data_ref = analysis_ref.data

# crystal_ref = CRYSTALS[meta_ref["material"]]()

# if crystal_ref.axiality == "uniaxial":
#     from fitting_strategies.jerphagnon1970 import Jerphagnon1970Strategy
#     strategy_ref = Jerphagnon1970Strategy(analysis_ref)

# elif crystal_ref.axiality == "biaxial":
#     from fitting_strategies.ishidate1974 import Ishidate1974Strategy
#     strategy_ref = Ishidate1974Strategy(analysis_ref)

In [ ]:
fit, aux = strategy._maker_fringes(override={"L":L, "theta_deg": position_centered}, return_aux=True)
d_factor = aux["d_factor"]

    # # ref
    # peak_ref = 1.286
    # _, aux_ref = strategy_ref._maker_fringes(return_aux=True)
    # d_factor_ref = aux_ref["d_factor"]
    # d_propto_ref = np.sqrt((1.0/0.11) * peak_ref / d_factor_ref)

    # ratio = d_propto/d_propto_ref 

    # d_ref_abs = 0.30 # d11 from shoji, et.al.
    # d_32_BMF = d_ref_abs * ratio

# BMF d32, 1 V/10 mV, OD2.0
d_propto = np.sqrt(0.01 * 1.0 * k / d_factor)

# BMFd31, 1/0.1 V, OD1+1+3.0
# d_propto = np.sqrt(0.1 * 1000 * 0.07 / d_factor)
ratio = d_propto/d_propto_quartz_wedge 

d_32_BMF = d_abs_quartz * ratio

print(aux["t"])
print(aux["T"])
print(f"ratio_to_quartz:{ratio}")
print(f"d_factor:{d_factor} ,d_32_BMF:{d_32_BMF}")

### Graphs on demand

In [ ]:
from matplotlib.ticker import ScalarFormatter

plt.rcParams['font.size'] = 20
fig, ax = plt.subplots(figsize=(12, 6))

norm_to_quartz = peak_quartz_wedge * 20.0 * (1/0.1176)

ax.plot(position_centered, (intensity_corrected - offset)/norm_to_quartz ,'-*', label='Experimental data')
ax.plot(position_centered, fitted_fringe/norm_to_quartz, label='Fitting')
# ax.plot(position_centered, fitted_envelope, label='Envelope')

ax.set_xlabel(r'Incident angle $\theta$ (deg.)', fontsize=22)
ax.set_ylabel('Intensity (a.u.)', fontsize=22)

# Force scientific notation on y-axis
ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
# Use math text (×10^{-3} instead of 1e−3)
formatter = ScalarFormatter(useMathText=True)
formatter.set_powerlimits((0, 0))
ax.yaxis.set_major_formatter(formatter)
ax.yaxis.get_offset_text().set_fontsize(22)

ax.tick_params(axis="both", labelsize=22)

# ax.set_xlim(-20, 20)
if np.max(fitted_envelope) > 10.0:
      ax.set_ylim(0, np.max(intensity_corrected)*1.1)

fig.tight_layout()
ax.legend(loc='upper left', fontsize=20)
plt.show()

### minima/maxima detection trial

In [ ]:
x = data["position_centered"]
y = data["offset_corrected"]

try:
    if x is None or y is None:
        raise RuntimeError("x/y is None.")

    period, idx, freq, fft = strategy.estimate_fringe_period(x, y)
    fft = np.abs(fft)

    print(f"estimated index: {idx} period :{period}")

    plt.figure()
    plt.plot(1/freq, fft, "-")
    plt.plot(1/freq[idx], fft[idx], "*", color="r")
    plt.xlabel("Period [deg]")
    plt.ylabel("FFT amplitude")
    plt.title("FFT of fringe data")
    # plt.xlim(0, 0.1)
    # plt.yscale("log")
    plt.grid(True)
    plt.show()

    # Also print an approximate window length suggestion, if period is valid
    if period is not None:
        dx = float(np.median(np.diff(x)))
        window = int((period / dx) / 3)
        if window < 5:
            window = 5
        if window % 2 == 0:
            window += 1
        print(f"dx ~ {dx:.6g} deg, suggested window length ~ {window} samples")

except Exception as e:
    print("estimate_fringe_period failed:", repr(e))

### Theoretical fringe for different thickness

In [ ]:
meta = analysis.meta
data = analysis.data

x = analysis.data["position"]
intensity_corrected = analysis.data["intensity_corrected"]

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.size'] = 10
fig, ax = plt.subplots(figsize=(12, 4))

for i in [3.8]:
    fringe_on_L = strategy._maker_fringes(override={"L":i})
    # envelope = strategy._maker_fringes(override={"L":i}, envelope=True)
    ax.plot(x, fringe_on_L, label=f'L= {i}')
    # ax.plot(x, envelope, label=f'envelope at L={i}')

ax.set_xlabel(r'$\theta$ (deg.)', fontsize=12)
ax.set_ylabel('Intensity (a.u.)', fontsize=12)

# ax.set_xlim(-20, 20)
ax.set_ylim(0, 3)

fig.tight_layout()
ax.legend(loc='upper left')
plt.show()

In [ ]:
x = np.linspace(1.5, 2.0, 50)
y = []
for i in x:
    fringe = strategy._maker_fringes(override={"L":i, "theta_deg":0})
    y.append(fringe)

plt.plot(x, y, "-*")
plt.grid(True)
plt.show()

print(np.max(y))